In [87]:
import numpy as np
import pandas as pd

In [90]:
t_d = pd.read_csv('train_data.csv')
t_a = pd.read_csv('train_answers.csv')
t_gd = pd.read_csv('train_data_gd.csv')
t_d_a = t_d.merge(t_a, on=['itemId'])
t_d_a = t_d_a.merge(t_gd, on=['itemId'])
true_column_suffix = '_gd'
columns = ['area', 'L', 'W', 'X', 'Y']
true_columns = list(map(lambda x: x + true_column_suffix, columns))
for i in ['', true_column_suffix]:
    t_d_a['L' + i] = t_d_a['Xmax' + i] - t_d_a['Xmin' + i]
    t_d_a['W' + i] = t_d_a['Ymax' + i] - t_d_a['Ymin' + i]
    t_d_a['area' + i] = t_d_a['L' + i] * t_d_a['W' + i]
    t_d_a['X' + i] = (t_d_a['Xmax' + i] + t_d_a['Xmin' + i])/2
    t_d_a['Y' + i] = (t_d_a['Ymax' + i] + t_d_a['Ymin' + i])/2

In [94]:
def avg(a, b):
    return (a+b)/2
    
def position_score(line, theta):
    global true_column_suffix    
    W = max(1, max(line['L'], line['L' + true_column_suffix]))
    H =  max(1, max(line['W'], line['W' + true_column_suffix]))
    X = abs(line['X'] - line['X' + true_column_suffix])
    Y = abs(line['Y'] - line['Y' + true_column_suffix])
    return 1 - avg(X/W/(1 - theta), Y/H/(1 - theta))

def size_score(line, theta):
    global true_column_suffix
    a,b = line['area'], line['area' + true_column_suffix]
    if a < b:
        b,a = a,b
    if a == 0:
        return 0        
    return 1 - (1 - b/a)/(1 - theta)

def shape_score(line, theta):
    global true_column_suffix
    ral,raw = line['L'], max(1, line['W'])
    rbl,rbw = line['L' + true_column_suffix], max(1, line['W' + true_column_suffix])
    ra = ral/raw
    rb = rbl/rbw
    a = ra/max(1,rb)
    if ra > rb :
        a = rb/max(1,ra)
    return 1 - (1 - a)/(1 - theta)

In [101]:
theta = .5
t_d_a['size'] = t_d_a.apply(size_score, axis=1, theta=theta)
t_d_a['shape'] = t_d_a.apply(shape_score, axis=1, theta=theta)
t_d_a['position'] = t_d_a.apply(position_score, axis=1, theta=theta)
t_d_a['score'] = t_d_a[['size', 'shape', 'position']].apply(np.mean, axis=1)

In [102]:
t_d_a[columns + true_columns + ['size', 'position', 'shape', 'score']]

,area,L,W,X,Y,area_gd,L_gd,W_gd,X_gd,Y_gd,size,position,shape,score
0,31458,214,147,169.0,729.5,32364,186,174,164.0,734.0,0.944012,0.950773,0.468579,0.787788
1,3978,51,78,209.5,701.0,32364,186,174,164.0,734.0,-0.754171,0.565721,0.223325,0.011625
2,38038,209,182,167.5,739.0,32364,186,174,164.0,734.0,0.701667,0.955781,0.861739,0.839729
3,27548,142,194,182.0,734.0,32364,186,174,164.0,734.0,0.702385,0.903226,0.369471,0.658361
4,30576,182,168,151.0,735.0,32364,186,174,164.0,734.0,0.889507,0.924360,0.973475,0.929114
5,36960,210,176,144.0,737.0,32364,186,174,164.0,734.0,0.751299,0.887716,0.791790,0.810268
6,9118,97,94,77.5,440.0,7290,90,81,70.0,437.5,0.599035,0.896085,0.857447,0.784189
7,8170,95,86,72.5,442.0,7290,90,81,70.0,437.5,0.784578,0.921359,0.988372,0.898103
8,6059,83,73,66.5,435.5,7290,90,81,70.0,437.5,0.662277,0.936420,0.954485,0.851060
9,101400,325,312,371.5,895.0,116382,357,326,371.5,895.0,0.742538,1.000000,0.902428,0.881655
